<a href="https://colab.research.google.com/github/sarikasea/Python/blob/main/2_Amazon_Medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns



In [2]:

!pip install --upgrade gspread gspread_dataframe
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials

from gspread_dataframe import get_as_dataframe
from google.colab import auth
auth.authenticate_user()
import gspread

In [3]:
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# Amazon - Medium

In [4]:
sheet = gc.open('Python Summer Challenge').worksheet('Q2-1')
dim_product = get_as_dataframe(sheet)
dim_product

,product_id,product_name,product_category
0,1.0,Smart TV,Home Electronics
1,2.0,Wireless Earbuds,Electronics & Gadgets
2,3.0,Refrigerator,Electronics Appliances
3,4.0,Bestselling Novel,Books
4,5.0,Designer Jeans,Fashion
5,6.0,Blender,Kitchen
6,7.0,Tent,Outdoor
7,8.0,Smart Home Hub,Home Electronics
8,9.0,Phone Charger,Electronics Accessories
9,10.0,Skincare Set,Health & Beauty


In [5]:
sheet = gc.open('Python Summer Challenge').worksheet('Q2-2')
fct_ad_performance = get_as_dataframe(sheet)
fct_ad_performance

,ad_id,clicks,product_id,impressions,recorded_date
0,101.0,10.0,1.0,200.0,2024-10-02
1,102.0,15.0,1.0,300.0,2024-10-12
2,103.0,20.0,2.0,250.0,2024-10-05
3,104.0,18.0,2.0,230.0,2024-10-20
4,105.0,5.0,3.0,150.0,2024-10-15
5,106.0,12.0,3.0,180.0,2024-10-25
6,107.0,50.0,4.0,500.0,2024-10-07
7,108.0,8.0,5.0,250.0,2024-10-18
8,109.0,14.0,6.0,200.0,2024-10-10
9,110.0,22.0,8.0,220.0,2024-10-30


What is the average click-through rate (CTR) for sponsored product ads for each product category that contains the substring 'Electronics' in its name during October 2024? This analysis will help determine which electronics-related categories are performing optimally.


Tables
Explore data
fct_ad_performance(ad_id, product_id, clicks, impressions, recorded_date)
dim_product(product_id, product_category, product_name)

In [6]:
# Note: # Merge the two dataframes on product_id


merged_df = pd.merge(dim_product, fct_ad_performance, on='product_id')

# Filter for product categories containing 'Electronics'
electronics_df = merged_df[merged_df['product_category'].str.contains('Electronics', case=False, na=False)]

# Convert 'recorded_date' to datetime and filter for October 2024
electronics_df['recorded_date'] = pd.to_datetime(electronics_df['recorded_date'])
october_electronics_df = electronics_df[(electronics_df['recorded_date'].dt.year == 2024) & (electronics_df['recorded_date'].dt.month == 10)]

# Aggregate total clicks and impressions for each product category
aggregated_electronics_df = october_electronics_df.groupby('product_category').agg({
    'clicks': 'sum',
    'impressions': 'sum'
}).reset_index()

# Calculate the overall CTR for each product category using aggregated values
# Avoid division by zero by replacing 0 impressions with a small number or handling them
aggregated_electronics_df['Overall_CTR'] = (aggregated_electronics_df['clicks'] / aggregated_electronics_df['impressions']) * 100
aggregated_electronics_df['Overall_CTR'] = aggregated_electronics_df['Overall_CTR'].replace([np.inf, -np.inf], 0) # Handle potential division by zero if impressions were 0

# Display the result
print("Overall CTR for Electronics-related categories in October 2024:")
merged_df = pd.merge(df2, df1, on='product_id')

# Convert 'recorded_date' to datetime and filter for October 2024
merged_df['recorded_date'] = pd.to_datetime(merged_df['recorded_date'])
october_df = merged_df[(merged_df['recorded_date'].dt.year == 2024) & (merged_df['recorded_date'].dt.month == 10)]
(aggregated_electronics_df[['product_category', 'Overall_CTR']])

Overall CTR for Electronics-related categories in October 2024:


/tmp/ipython-input-1284272259.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  electronics_df['recorded_date'] = pd.to_datetime(electronics_df['recorded_date'])


,product_category,Overall_CTR
0,Electronics & Gadgets,7.916667
1,Electronics Accessories,10.000000
2,Electronics Appliances,5.151515
3,Electronics Gadgets,7.407407
4,Home Electronics,6.527778


Which product categories have a CTR greater than the aggregated overall average CTR for sponsored product ads during October 2024? This analysis will identify high-performing categories for further optimization. For this question, we want to calculate CTR for each ad, then get the average across ads by product category & overall.

In [10]:
# Merge the two dataframes on product_id
merged_df = pd.merge(dim_product, fct_ad_performance, on='product_id')

# Convert 'recorded_date' to datetime and filter for October 2024
merged_df['recorded_date'] = pd.to_datetime(merged_df['recorded_date'])
october_df = merged_df[(merged_df['recorded_date'].dt.year == 2024) & (merged_df['recorded_date'].dt.month == 10)]


# Calculate CTR for each ad
october_df['CTR'] = (october_df['clicks'] / october_df['impressions']) * 100
october_df['CTR'] = october_df['CTR'].replace([np.inf, -np.inf], 0) # Handle potential division by zero

# Calculate the average CTR for each product category
average_ctr_by_category = october_df.groupby('product_category')['CTR'].mean().reset_index()
average_ctr_by_category.rename(columns={'CTR': 'Average_Ad_CTR'}, inplace=True)

# Calculate the overall average CTR across all ads
overall_average_ctr = october_df['CTR'].mean()

# Filter for product categories with average CTR greater than the overall average
high_performing_categories = average_ctr_by_category[average_ctr_by_category['Average_Ad_CTR'] > overall_average_ctr]

print(f"Overall Average CTR across all ads in October 2024: {overall_average_ctr:.2f}%")
print("\nProduct categories with Average Ad CTR greater than the Overall Average CTR:")
print(high_performing_categories)

Overall Average CTR across all ads in October 2024: 6.80%

Product categories with Average Ad CTR greater than the Overall Average CTR:
          product_category  Average_Ad_CTR
1                    Books       10.000000
2    Electronics & Gadgets        7.913043
3  Electronics Accessories       10.000000
5      Electronics Gadgets        7.250000
8                  Kitchen        7.000000


For the product categories identified in the previous question, what is the percentage difference between their CTR and the overall average CTR for October 2024? This analysis will quantify the performance gap to recommend specific categories for targeted advertising optimization.

In [12]:
# Calculate the percentage difference for each high-performing category
high_performing_categories['Percentage_Difference_vs_Overall'] = (
    (high_performing_categories['Average_Ad_CTR'] - overall_average_ctr) / overall_average_ctr
) * 100

print("\nPercentage Difference in Average Ad CTR vs. Overall Average CTR for High-Performing Categories:")
print(high_performing_categories[['product_category', 'Percentage_Difference_vs_Overall']])


Percentage Difference in Average Ad CTR vs. Overall Average CTR for High-Performing Categories:
          product_category  Percentage_Difference_vs_Overall
1                    Books                         46.961679
2    Electronics & Gadgets                         16.291416
3  Electronics Accessories                         46.961679
5      Electronics Gadgets                          6.547217
8                  Kitchen                          2.873175


/tmp/ipython-input-2453429819.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_performing_categories['Percentage_Difference_vs_Overall'] = (
